# Deep Q-Learning for Lunar Landing

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium
- https://pypi.org/project/gymnasium/
- https://pypi.org/project/swig/

In [ ]:
#!pip install gymnasium
#!pip install "gymnasium[atari, accept-rom-license]"
#!apt-get install -y swig
#!pip install gymnasium[box2d]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


### Importing the libraries

In [2]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple
import gymnasium
torch.cuda.is_available()

True

## Part 1 - Building the AI

### Creating the architecture of the Neural Network
- 3 fully connected layers
- input of dim spaces
- output of dim actions

In [3]:
class Network(nn.Module):

  def __init__(self, state_size, action_size, seed = 42):
    super(Network, self).__init__()
    self.seed = torch.manual_seed(seed)
    self.fc1 = nn.Linear(state_size, 64)
    self.fc2 = nn.Linear(64, 64)
    self.fc3 = nn.Linear(64, action_size)

  def forward(self, state):
    x = self.fc1(state)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    return self.fc3(x)

## Part 2 - Training the AI

### Setting up the environment

In [4]:

env = gymnasium.make('LunarLander-v2')
# get environment key parameters from env
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n
# print key metrics
print('State shape: ', state_shape)
print('State size: ', state_size)
print('Number of actions: ', number_actions)

State shape:  (8,)
State size:  8
Number of actions:  4


### Initializing the hyperparameters

In [5]:
learning_rate = 5e-4
minibatch_size = 100
discount_factor = 0.99              # closer to one makes rewards of future actions count
replay_buffer_size = int(1e5)       # number of experiemnts to store in the memory Replay
interpolation_parameter = 1e-3

### Implementing Experience Replay

In [6]:
class ReplayMemory(object):

  def __init__(self, capacity):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.capacity = capacity
    self.memory = []

  def push(self, event):
    # add experiments to the buffer while keeping buffer size below max size
    self.memory.append(event)
    if len(self.memory) > self.capacity:
      del self.memory[0]

  def sample(self, batch_size):
    # randomly select n = batch_size experiments from the memory buffer
    experiences = random.sample(self.memory, k = batch_size)
    # each experiment contains the state, the action, the reward, the next_state and the flag done
    # stack the information only if the experiment exists (not None)
    # convert to tensors with float datatype, except actions which are 0 to 3 and dones boolean type
    states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
    actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
    rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
    next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
    dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)
    return states, next_states, actions, rewards, dones

### Implementing the DQN class

In [7]:
class Agent():

  def __init__(self, state_size, action_size):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.state_size = state_size
    self.action_size = action_size
    # create two Q networks, the internal and the external ones, which stabilize the learning process
    self.local_qnetwork = Network(state_size, action_size).to(self.device)      # internal Q network select the actions
    self.target_qnetwork = Network(state_size, action_size).to(self.device)     # external Q network computes q values and used to train the internal network
    # training algo
    self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr = learning_rate)
    # replay memory
    self.memory = ReplayMemory(replay_buffer_size)
    # time step counter
    self.t_step = 0

  def step(self, state, action, reward, next_state, done):
    # add experiment to memory
    self.memory.push((state, action, reward, next_state, done))
    # increment timestep and learn every 4 steps (reset timestep)
    self.t_step = (self.t_step + 1) % 4
    # every 4 steps, sample minibatch  of experiences, learn from the batch
    if self.t_step == 0:
      if len(self.memory.memory) > minibatch_size:
        experiences = self.memory.sample(minibatch_size)
        self.learn(experiences, discount_factor)

  # method to select the best action
  def act(self, state, epsilon = 0.):
    # convert to tensor, add one dim to account for the batch in position 0
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
    # forward pass, get action q values
    self.local_qnetwork.eval()
    with torch.no_grad():
      action_values = self.local_qnetwork(state)
    # select predicted action using greedy policy
    self.local_qnetwork.train()
    if random.random() > epsilon:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))

  def learn(self, experiences, discount_factor):
    # unpacking
    states, next_states, actions, rewards, dones = experiences
    # action values prediction using target network, add dimension in position 1
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + discount_factor * next_q_targets * (1 - dones)
    # get expected q values by propagate using internal network
    q_expected = self.local_qnetwork(states).gather(1, actions)
    # compute loss
    loss = F.mse_loss(q_expected, q_targets)
    # backpropagate the loss to compute gradient
    self.optimizer.zero_grad()
    loss.backward()
    # update parameters
    self.optimizer.step()
    # update target network with internal network weights
    self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_parameter)

  # update parameters between networks during training step slowly to avoid learning destabilization
  def soft_update(self, local_model, target_model, interpolation_parameter):
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)

### Initializing the DQN agent

In [14]:
agent = Agent(state_size, number_actions)

### Training the DQN agent
- winning score see here: https://gymnasium.farama.org/environments/box2d/lunar_lander/

In [15]:
# max number of episodes over which to train agent
number_episodes = 2000
# avoid spending too much time on one episode, max 1000 timesteps in one episode
maximum_number_timesteps_per_episode = 1000
# for epsilon greedy selection of action
epsilon_starting_value  = 1.0
epsilon_ending_value  = 0.01
epsilon_decay_value  = 0.995
epsilon = epsilon_starting_value
# window of scores on last 100 episodes using queue
scores_on_100_episodes = deque(maxlen = 100)

# Training Loop
for episode in range(1, number_episodes + 1):
  state, _ = env.reset()
  score = 0
  for t in range(maximum_number_timesteps_per_episode):
    action = agent.act(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)
    state = next_state
    score += reward
    if done:
      break
  # store score to queue
  scores_on_100_episodes.append(score)
  # update epsilon decay
  epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end = "")
  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
  # check if we reach 200 when we consider training finished because of the reward for landing (see gymnasium page of LunarLander)
  if np.mean(scores_on_100_episodes) >= 200.0:
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
    break

Episode 100	Average Score: -174.33
Episode 200	Average Score: -94.941
Episode 300	Average Score: -32.11
Episode 400	Average Score: 26.914
Episode 500	Average Score: 92.57
Episode 600	Average Score: 115.13
Episode 700	Average Score: 158.69
Episode 757	Average Score: 201.23
Environment solved in 657 episodes!	Average Score: 201.23


In [12]:
# no training performed - load saved one
try:
    agent.local_qnetwork.load_state_dict(torch.load('checkpoint.pth'))
except:
    print('No saved model found')

## Part 3 - Visualizing the results

In [20]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gymnasium.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env_name):
    env = gymnasium.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('lunar_video.mp4', frames, fps=30)
    imageio.mimsave('lunar.gif', frames, duration=3)

show_video_of_model(agent, 'LunarLander-v2')

def show_video():
    mp4list = glob.glob('lunar_video.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
